In [15]:
import yahoo_fin.stock_info as si
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from pandas import read_html
#import FunctionFile as FF
from ipynb.fs.full.FunctionFile import RiskVal_func
from ipynb.fs.full.FunctionFile import stock_monte_carlo_func
from ipynb.fs.full.FunctionFile import GrahamInVal_func
from ipynb.fs.full.FunctionFile import WACC_func
from ipynb.fs.full.FunctionFile import EPSMult_func
from ipynb.fs.full.FunctionFile import DCFInVal_func

nan=np.nan

In [16]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

from pandas_datareader import data
from datetime import datetime

In [17]:
#****START HERE*****
#****Computes Graham Intrinsic Value, calls GrahamInVal_func function from FunctionFile
#****Inputs to provide : stock_list
#                        DBGmode (0 by default)
#****Outputs in stock_sum dataframe

stock_list = ['RGEN']
DBGmode=0

stock_sum = pd.DataFrame(np.arange(7*len(stock_list)).reshape((len(stock_list),7)),
                                   columns=['Sticker',
                                            'GrahamInVal($)',
                                            'PrevClose($)',
                                            'Decision', 
                                            'Margin(%)',
                                            'EPS',
                                            'GrowthEst(%)'])

for i in stock_list:
    ami = GrahamInVal_func(i,DBGmode)
    ami1 = pd.DataFrame.from_dict(ami, orient='index')
    ami2 = ami1.transpose()
    stock_sum = stock_sum.append(ami2)

stock_sum.set_index('Sticker', inplace=True)
stock_sum.drop(stock_sum.index[0:len(stock_list)], inplace=True)
stock_sum

,GrahamInVal($),PrevClose($),Decision,Margin(%),EPS,GrowthEst(%)
Sticker,,,,,,
RGEN,37.96,150.87,SELL,297.44,0.6,26.5


In [18]:
#****Computes Weighted Average Cost of Capital (WACC), calls WACC_func function from FunctionFile
#****Inputs to provide : stock_list
#                        DBGmode (0 by default)
#****Outputs in stock_anal0 = concatenated WACC_df & stock_sum
DBGmode=0
WACC_df = pd.DataFrame(columns = ['WACC'], index = stock_list)

for x in stock_list:
    WACC_df['WACC'][x] = WACC_func(x,DBGmode)

stock_anal0=pd.concat([stock_sum, WACC_df], axis=1, sort=False)
stock_anal0

,GrahamInVal($),PrevClose($),Decision,Margin(%),EPS,GrowthEst(%),WACC
Sticker,,,,,,,
RGEN,37.96,150.87,SELL,297.44,0.6,26.5,0.0942052


In [19]:
#****Computes Intrisic Value by EPS Multiplier Method, calls EPSMult_func function from FunctionFile
#****Inputs to provide : stock_list
#                        eps (from stock_anal0)
#                        growthest (from stock_anal0)
#                        wacc (from stock_anal0)
#                        DBGmode (0 by default)
#****Outputs in stock_anal1 = concatenated EPSMult_df & stock_anal0

DBGmode=0
EPSMult_df = pd.DataFrame(columns = ['EPSMult_InVal'], index = stock_list)

for i in stock_list:
    EPSMult_df['EPSMult_InVal'][i] = EPSMult_func(stock_anal0['EPS'][i],stock_anal0['GrowthEst(%)'][i],stock_anal0['WACC'][i],DBGmode)

stock_anal1=pd.concat([stock_anal0, EPSMult_df], axis=1, sort=False)
stock_anal1

,GrahamInVal($),PrevClose($),Decision,Margin(%),EPS,GrowthEst(%),WACC,EPSMult_InVal
Sticker,,,,,,,,
RGEN,37.96,150.87,SELL,297.44,0.6,26.5,0.0942052,56.806


In [24]:
#****Computes Intrisic Value by Discounted Cash Flow Method, calls DCFInVal_func function from FunctionFile
#****Inputs to provide : stock ticker
#                        GRAVG1LAST0 (1 for average, 0 for last yr)
#                        wacc (from stock_anal0)
#                        DBGmode (0 by default)
#****Outputs in stock_anal2 = concatenated DCFInVal_df & stock_anal1

DBGmode=0
GRAVG1LAST0=0
DCFInVal_df = pd.DataFrame(columns = ['DCFInVal'], index = stock_list)

for i in stock_list:
    DCFInVal_df['DCFInVal'][i] = DCFInVal_func(i,GRAVG1LAST0,stock_anal1['WACC'][i],DBGmode)

stock_anal2=pd.concat([stock_anal1, DCFInVal_df], axis=1, sort=False)
stock_anal2

,GrahamInVal($),PrevClose($),Decision,Margin(%),EPS,GrowthEst(%),WACC,EPSMult_InVal,DCFInVal
Sticker,,,,,,,,,
RGEN,37.96,150.87,SELL,297.44,0.6,26.5,0.0942052,56.806,61.16


In [21]:
#****Computes Value at Risk, calls RiskVal_func function from FunctionFile
#****Inputs to provide : stock_list
#                       lookbackperiod (in years)
#****Outputs in stock_anal3 = concatenated risk_df & stock_anal2

lookbackperiod=1
risk_df = RiskVal_func(stock_list,lookbackperiod)

stock_anal3=pd.concat([stock_anal2, risk_df], axis=1, sort=False)
stock_anal3

,GrahamInVal($),PrevClose($),Decision,Margin(%),EPS,GrowthEst(%),WACC,EPSMult_InVal,DCFInVal,1yr/95% WC Risk%,1yr/99% WC Risk%
Sticker,,,,,,,,,,,
RGEN,37.96,150.87,SELL,297.44,0.6,26.5,0.0942052,56.806,61.16,-4.281319,-8.964574


In [22]:
#****Computes Monte Carlo Value at Risk, calls stock_monte_carlo_func function from FunctionFile
#****Inputs to provide : stock_list
#                        lookbackperiod (in years) defined above for RiskVal
#                        lookaheadperiod (in days) for stock estimate
#                        runs (for MC run size)
#****Outputs in stock_anal4 = concatenated MC_df & stock_anal3

# lookbackperiod=1 (defined already above)
lookaheadperiod=365
runs = 10000
days=lookaheadperiod

end = datetime.now()
start = datetime(end.year-lookbackperiod, end.month, end.day)
for x in stock_list:
    globals()[x] = data.DataReader(x,'yahoo',start,end)
        
closing_df = data.DataReader(stock_list,'yahoo',start,end)['Adj Close']
tech_rets = closing_df.pct_change()
rets = tech_rets.dropna()

MC_df = pd.DataFrame(columns = ['start_price','mu','sigma','1yr/99%/q','Var_WC','1Yr/99%/Risk%'], index = stock_list)

for i in stock_list:
    MC_df['mu'][i] = rets.mean()[i]
    MC_df['sigma'][i] = rets.std()[i]
    MC_df['start_price'][i] = float(closing_df.tail(1)[i])
        
    simulations = np.zeros(runs)

    for run in range(runs):
        simulations[run]=stock_monte_carlo_func(MC_df['start_price'][i],days,MC_df['mu'][i],MC_df['sigma'][i])[days-1]
    
    MC_df['1yr/99%/q'][i] = np.percentile(simulations,1)
    MC_df['Var_WC'][i] = MC_df['1yr/99%/q'][i]-MC_df['start_price'][i]
    MC_df['1Yr/99%/Risk%'][i] = 100*(MC_df['1yr/99%/q'][i]-MC_df['start_price'][i])/MC_df['start_price'][i]

stock_anal4=pd.concat([stock_anal3, MC_df], axis=1, sort=False)
stock_anal4.drop(columns=['mu','sigma','Var_WC'],inplace=True)
stock_anal4

,GrahamInVal($),PrevClose($),Decision,Margin(%),EPS,GrowthEst(%),WACC,EPSMult_InVal,DCFInVal,1yr/95% WC Risk%,1yr/99% WC Risk%,start_price,1yr/99%/q,1Yr/99%/Risk%
Sticker,,,,,,,,,,,,,,
RGEN,37.96,150.87,SELL,297.44,0.6,26.5,0.0942052,56.806,61.16,-4.281319,-8.964574,151.64,142.248,-6.19371
